## Math Tutor

Reference:
- [OpenAI Reference Sample - Math Tutor](https://github.com/openai/openai-python/blob/main/examples/assistant.py)

In [ ]:
import time
import os
from dotenv import load_dotenv

from openai import AzureOpenAI

# gets API Key from environment variable OPENAI_API_KEY
load_dotenv()
api_endpoint = os.getenv("OPENAI_URI")
api_key = os.getenv("OPENAI_KEY")
api_version = os.getenv("OPENAI_VERSION")
api_deployment_name = os.getenv("OPENAI_GPT_DEPLOYMENT")


client = AzureOpenAI(api_key=api_key,
api_version=api_version,
azure_endpoint=api_endpoint)

In [ ]:
assistant = None
print(api_deployment_name)
try:
    assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    tools=[{"type": "code_interpreter"}],
    model=api_deployment_name,
)
except Exception as e:
    print(e)

thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?",
)

run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Please address the user as Jane Doe. The user has a premium account.",
)

print("checking assistant status. ")
while True:
    run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

    if run.status == "completed":
        print("done!")
        messages = client.beta.threads.messages.list(thread_id=thread.id)

        print("messages: ")
        for message in messages:
            assert message.content[0].type == "text"
            print({"role": message.role, "message": message.content[0].text.value})

        # Clean up
        client.beta.assistants.delete(assistant.id)
        client.beta.threads.delete(thread.id)

        break
    else:
        print("in progress...")
        time.sleep(5)